## SI 670 Applied Machine Learning, Week 5:  One-hot encoding, calibration, decision trees, random forests, data leakage. (Due Wednesday October 5 2022, 11:59pm.)

Total: 100 points
Question 1: 30 points
Question 2: 20 points
Question 3: 30 points 
Question 4: 20 points

* This homework is worth 100 points in total. Correct answers and code receive full credit, but partial credit will be awarded if you have the right idea even if your final answers aren't quite right.

* Submit your completed notebook file to the Canvas site - **IMPORTANT**: please name your submitted file `si670f22-hw5-youruniqname.ipynb`

* Any file submitted after the deadline will be marked as late. Please consult the syllabus regarding late submission policies. You can submit the homework as many time as you want, but only your latest submission will be graded.

* As a reminder, the notebook code you submit must be your own work. Feel free to discuss general approaches to the homework with classmates. If you end up forming more of a team discussion on multiple questions, please include the names of the people you worked with at the top of your notebook file.


### Question 1 (30 points)

This question doesn't require coding, in the sense of writing programs that do all the computing of the answer for you: you can figure out the answer by hand and then put your answers into the notebook by defining the correct numpy array in python. Alternatively you could write out your answers in markdown using simple LaTeX tags. (See [here](https://csrgxtu.github.io/2015/03/20/Writing-Mathematic-Fomulars-in-Markdown/) for how to write things with math operators in LaTeX format.)


#### (a) (10 points) One-hot encoding

If you have a dataset with three data points, and each data point has three features. Among them, X2 and X3 are categorical variables:

|    X1	|  X2 	|  X3	|
|----	|----	|----	|
|   1.3	|  a 	| a 	|
|   0.7 |  b 	| c 	|
|   2.1 |  a 	| b     |

X2 can take up the values a and b, and X3 can take up the values a, b and c.

Please manually convert this dataset into numerical format with the categorical variables transformed into one-hot encoding. Please keep the order of X1, X2, and X3, and use alphabetical order for the one-hot encoding.


#### (b) (10 points) Calibration
Recall the calibration curve has the predicted probability as its x-axis and the true probability (also known as the "empirical probability" as its y-axis. Suppose you are given a binary classifier and its predicted probabilities on a test set with 15 data points. The labels of these data points are also given. Please calculate the true probabilities in three bins: \[0, 0.3), \[0.3, 0.7), \[0.7, 1\]. You could further use these probabilities to draw a calibration curve but it's not required for this question. You only need to give the 3 numbers indicating the true probabilities for each bin.  It might help to recall the weather example from class: the "true" empirical probability of rain is just the fraction of times it *actually* rained according to the data, for a given predicted probability (or range of probabilities) from the weatherperson.

|Predicted probability | Label |
|----	               |----   |
|   0.40               |   0   |
|   0.77               |   1   |
|   0.84               |   0   |
|   0.68               |   0   |
|   0.73               |   1   |
|   0.88               |   1   |
|   0.69               |   0   |
|   0.24               |   0   |
|   0.70               |   1   |
|   0.41               |   1   |
|   0.34               |   1   |
|   0.18               |   1   |
|   0.31               |   1   |
|   0.58               |   1   |
|   0.00               |   0   |


#### (c) (10 points) Random forest parameters

Suppose your current random forest classifier is facing an overfitting situation. Please state whether increasing or decreasing the following parameters can potentially help you reduce overfitting or not, and why?

(i) `n_estimators`

(ii) `max_features`

(iii) `max_depth`

(iv) `n_jobs`


In [6]:
import numpy as np
df = np.array([1.3, 0.7, 2.1])
df

array([1.3, 0.7, 2.1])

In [8]:
# (a) 
#just create columns X2_a and X2_b and then X3_a,X3_b and X3_c. Drop one column because of correlation

#replace {a :1, b:0} and rename column to X2_a
#create new columns X3_a,X3_b and enter value 1 in X3_a if the value in X3 is a otherwise 0, do the same 
# for X3_b and then just drop X3

# X1 = np.array(1.3, 0.7, 2.1)
# df = 


**(c)**
(i) `n_estimators`: Increasing the value of this parameters reduces overfitting for larger datasets because on a small dataset there is only so many diferent trees we can fit till the prediction becomes stable but for a larger dataset the prediction is an average of the result produced by the different trees and hence, even if a individual tree is overfitting, the result is derived from combining result from all different trees. So, more the no.of decision trees more stable will be our prediction.

(ii) `max_features`: We want max-features to not be very small not a be the whole n-features because an optimum value will allow for the trees to be diverse, thus making the average prediction more meaningful. If we use very less features that might lead to inefficient tree (not able to partition properly) and if we use all the features, we will get similar trees and mit can lead to overfitting. So, the relationship is non-linear.

(iii) `max_depth` : Max depth does not effect overfitting in case of random forests unlike decision trees, because even if one tree overfits not all of them would, so the average prediction does not result in overfitting. 

(iv) `n_jobs`: This parameter is used for parallelization and does not effect overfitting in any way. More processors which just make the model run faster.


### Question 2 (20 points) Cross-validation for very small datasets.

In our lecture about data leakage, we talked about one simple strategy to help you avoid data leakage: before you do any work with a new dataset, split off a final test dataset, and use this final test dataset as the very last step in your validation. However, when you have a very small dataset, one issue is that it leads to really tiny test sets, which leads to unreliable test evaluation scores. For example, if you have a dataset with a total of 60 samples, and hold back 25% as a final test set, you'll get a final test set with 15 samples. In this case, a single evaluation score based on merely 15 samples could be very unreliable and probably not something to be relied on heavily.

To make evaluation more reliable, we discussed how people usually use *cross-validation* to generate *multiple* evaluation scores, each on a different train/test split of the data. That is, you split the train and test set multiple times and then calculate the average of the resulting test scores. This is the approach we'll use to estimate a more reliable final test set score.  We don't want to use these final test sets to also tune our hyperparameters (to avoid data leakage), so we make sure to learn the model and tune the hyperparameters using only the data in the training split.  To do that, we do a second split *within the training data split* so that we have (i) a training set for the model and (ii) a separate *validation split* that's used to evaluate the model and pick the best setting for the hyperparameters.

Here's the general recipe:

1. Split the whole dataset into $k$ equal folds
2. For $i$ from 1 to $k$   (for each of the $k$ folds)

    a. Take the $i$-th fold as a final test set. 
    
    b. With the remaining data (i.e. combining the other folds), apply a standard train/test split (75%/25%).
    
    c. For each possible tuning value of hyperparameter (in our case, alpha):
    
       - Train the model with the 75% part using a specific hyperparameter choice (for alpha)
             
       - Evaluate the model with the 25% part. This 25% is our "validation set".
             
       Pick the hyperparameter value that gave the best score on the validation set.
            
    d. Once you find the optimal hyperparameter, do the test set predictions on this $i$-th fold and calculate the test score for this $i$-th fold.
  
3. Report the average of the final test set scores you got across all $k$ folds.

To simulate a small dataset scenario, we have provided the code that selects the first 60 samples from the built-in boston dataset.. We've also given you the variable 'alpha_list', which has the range of ridge regression hyperparameter alphe you should use for tuning. 

Write the code that implements the above scheme on this subset of the boston dataset. You can split the whole dataset into k folds (step 1) by using the handy `KFold` function (https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html). For each fold, you can then use the default `train_test_split` to get that's fold's training and validation data (step 2b). With this training data, train a Ridge regression model, and use the validation set to evaluate, tuning to find the optimal hyper-parameter alpha value within each fold (step 2c). Get the final test set score using this optimal model (step 2d). Repeat for all $k$ folds to obtain a set of final test scores.

Finally, you need to return the mean value of the $k$ final test scores. This is your final (more reliable) test set prediction.

In [48]:
from sklearn.datasets import load_boston
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

X,y = load_boston(return_X_y=True)
X=X[:60,:]
y=y[:60]
alpha_list = [0.001,0.01,0.1,1,10]
    
# Your code here
kf = KFold(n_splits=5, shuffle =True, random_state=0)
a =kf.get_n_splits(X)
count = 0
score_list =[]
score_list_2 =[]
for train_index, test_index in kf.split(X):
    
#     print("TRAIN:", train_index, "TEST:", test_index)
#     print("size:",train_index.shape, test_index.shape )
    X_training, X_test = X[train_index], X[test_index]
    y_training, y_test = y[train_index], y[test_index]
    
    #further splitting the train data 75%/25%
    X_train, X_val,y_train, y_val = train_test_split(X_training, y_training, test_size = 0.25, random_state =0)
    
    scores =0
    best_alpha = 0
    best_model_2 = None
    for alpha_val in alpha_list:
        clf_ridge = Ridge(alpha = alpha_val).fit(X_train, y_train)
        val_score = clf_ridge.score(X_val, y_val)       
        if val_score > scores:
            best_alpha = alpha_val
            best_model_2 = clf_ridge
    print(best_alpha, val_score)
            
    score_list_2.append(best_model_2.score(X_test, y_test)) 
    
    
#     score_list.append(best_model.score(X_test, y_test))

    # trying all different alpha values
    ridge = Ridge()
    parameters = {'alpha':alpha_list}
    clf = GridSearchCV(ridge, parameters)
#     ridge_clf = clf.fit(X_train, y_train)
    ridge_clf = clf.fit(X_training, y_training)
    best_alpha =  ridge_clf.cv_results_['params'][ridge_clf.best_index_]['alpha']
#     print(best_alpha, ridge_clf.score(X_test, y_test))
    best_model = ridge_clf.best_estimator_

#     score_list.append(best_model.score(X_val, y_val))
    score_list.append(best_model.score(X_test, y_test))
        

sum(score_list)/ len(score_list), sum(score_list_2)/ len(score_list_2)
# score_list, score_list_2

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this case special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets 

10 0.7697830928625822
10 0.5766129330846955
10 0.7172525174722777
10 0.8077392115669034
10 0.61397744411235


(0.6489744786914632, 0.5212710351670162)

In [11]:
def answer_two(k):
    from sklearn.datasets import load_boston
    from sklearn.linear_model import Ridge
    from sklearn.model_selection import train_test_split
    from sklearn.model_selection import KFold

    X,y = load_boston(return_X_y=True)
    X=X[:60,:]
    y=y[:60]
    alpha_list = [0.001,0.01,0.1,1,10]
    
    # Your code here
    kf = KFold(n_splits=5, shuffle =True, random_state=0)

    score_list =[]
    for train_index, test_index in kf.split(X):
        X_training, X_test = X[train_index], X[test_index]
        y_training, y_test = y[train_index], y[test_index]
        
    
    return #mean_test_score
    
answer_two(3)    

TRAIN: [20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43
 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59] TEST: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 40 41 42 43
 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59] TEST: [20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39] TEST: [40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59]


### Question 3 (30 points)  Decision trees.

For this question, we'll work with the Statlog (German Credit Data) dataset that classifies people described by a set of attributes as good or bad credit risks. Download the dataset from [here](https://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data)) (Use the german.data). The last column is the prediction target and the remaining columns are features. 



(a) (20 points) First transform the categorical features into one-hot encodings. Then train a decision tree classifier and a random forest classifier. You should return 6 items as follows: the trained decision tree classifier, the trained random forest classifier, decision tree training accuracy, decision tree test accuracy,  random forest training accuracy, random forest test accuracy. **Please use random_state = 0 for train_test_split, decision tree classifier, and random forest classifier.**

*Hint 1: The columns of categorical features are 0, 2, 3, 5, 6, 8, 9, 11, 13, 14, 16, 18, 19.*

*Hint 2: You may have a problem using `OneHotEncoder` to handle string values and numerical values at the same time. You can transform the string columns first, and then concatenate with the numerical features.*



In [149]:
import pandas as pd
df = pd.read_csv('german.txt', sep=' ')
# df.select_dtypes(exclude = ['object'])
df.info(), 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 21 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   A11     999 non-null    object
 1   6       999 non-null    int64 
 2   A34     999 non-null    object
 3   A43     999 non-null    object
 4   1169    999 non-null    int64 
 5   A65     999 non-null    object
 6   A75     999 non-null    object
 7   4       999 non-null    int64 
 8   A93     999 non-null    object
 9   A101    999 non-null    object
 10  4.1     999 non-null    int64 
 11  A121    999 non-null    object
 12  67      999 non-null    int64 
 13  A143    999 non-null    object
 14  A152    999 non-null    object
 15  2       999 non-null    int64 
 16  A173    999 non-null    object
 17  1       999 non-null    int64 
 18  A192    999 non-null    object
 19  A201    999 non-null    object
 20  1.1     999 non-null    int64 
dtypes: int64(8), object(13)
memory usage: 164.0+ KB


(None,)

In [151]:
df['1.1'].unique()

array([2, 1])

In [150]:
df.columns

Index(['A11', '6', 'A34', 'A43', '1169', 'A65', 'A75', '4', 'A93', 'A101',
       '4.1', 'A121', '67', 'A143', 'A152', '2', 'A173', '1', 'A192', 'A201',
       '1.1'],
      dtype='object')

In [108]:
import numpy as np
target = '1.1'
X = df.drop(target, axis=1)
y= df[[target]]
num_cols = X.select_dtypes('number').columns
cat_cols = X.select_dtypes(exclude= np.number).columns
len(X.select_dtypes(exclude= np.number).columns), len([0, 2, 3, 5, 6, 8, 9, 11, 13, 14, 16, 18, 19])
# len(num_cols)

(13, 13)

In [116]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
# X_train[cat_cols.columns] = OneHotEncoder().fit_transform(X_train[cat_cols])
# X_train[num_cols.columns] = StandardScaler().fit_transform(X_train[num_cols])
# X_test[cat_cols.columns] =X_test_encoded = OneHotEncoder().fit_transform(X_test[cat_cols])
# X_test[num_cols.columns] = StandardScaler().fit_transform(X_test[num_cols])

# from sklearn.tree import DecisionTreeClassifier
# clf = DecisionTreeClassifier(max_features='sqrt', random_state=0)

# clf.fit(X_train, y_train)
# clf.score(X_test, y_test)

# X_train_encoded.categories_, #X_test_encoded.categories_
OH_cols_train = pd.DataFrame(OneHotEncoder(handle_unknown='ignore', sparse=False).fit_transform(X_train[cat_cols]))
OH_cols_train.index = X_train.index
# numeric_X_train = X_train.drop(cat_cols, axis=1)
# X_train[cat_cols]= 
# X_train[num_cols] = StandardScaler().fit_transform(X_train[num_cols])
OH_cols_train
# OneHotEncoder().fit_transform(X_train[cat_cols])

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
780,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
253,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
363,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
803,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
817,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
192,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
629,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
559,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0


In [157]:
 from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
df = pd.read_csv('german.txt', sep=' ')
    
target = '1.1'
X = df.drop(target, axis=1)
y= df[[target]]

num_attribs = list(X_train.select_dtypes(include=[np.number]))
cat_attribs = list(X_train.select_dtypes(exclude=[np.number]))

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
full_pipeline = ColumnTransformer([
            ("num", StandardScaler(), num_attribs),
            ("cat", OneHotEncoder(), cat_attribs),
        ])

train_X_prepared = full_pipeline.fit_transform(X_train)
test_X_prepared = full_pipeline.transform(X_test)

In [170]:
list(X_train.select_dtypes(include=[np.number]))
list(X_train.select_dtypes(exclude=[np.number]))

['A11',
 'A34',
 'A43',
 'A65',
 'A75',
 'A93',
 'A101',
 'A121',
 'A143',
 'A152',
 'A173',
 'A192',
 'A201']

In [172]:
def answer_three_a():
    import numpy as np
    from sklearn.compose import ColumnTransformer
    from sklearn.preprocessing import StandardScaler
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import accuracy_score
    import category_encoders as ce

    #load the downloaded dataset
    df = pd.read_csv('german.txt', sep=' ')
    
    target = '1.1'
    X = df.drop(target, axis=1)
    y= df[[target]]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
        # separate out numerical and string columns
        # do transformation like one -hot encoding and scaling.
        # tran decision tree and random forest


    num_attribs = list(X_train.select_dtypes(include=[np.number]))
    cat_attribs = list(X_train.select_dtypes(exclude=[np.number]))
    
    

    full_pipeline = ColumnTransformer([
            ("num", StandardScaler(), num_attribs),
            ("cat", OneHotEncoder(), cat_attribs),
        ])

#     train_X_prepared = full_pipeline.fit_transform(X_train)
#     test_X_prepared = full_pipeline.transform(X_test)
    
#     train_X_prepared = X_train
#     test_X_prepared = X_test

    encoder = ce.OneHotEncoder(cat_attribs)
    train_X_encoded = encoder.fit_transform(X_train)
    test_X_encoded = encoder.transform(X_test)
    
    scaler = StandardScaler()
    train_X_prepared = scaler.fit_tranform(X_train)
    test_X_prepared = scaler.transform(X_test)

    DT_classifier = DecisionTreeClassifier(max_depth=5, random_state=0)
    DT_classifier.fit(train_X_prepared, y_train)
    DT_y_pred_train = DT_classifier.predict(train_X_prepared)
    DT_y_pred_test = DT_classifier.predict(test_X_prepared)
    train_score_DT = accuracy_score(y_train, DT_y_pred_train)
    test_score_DT = accuracy_score(y_test, DT_y_pred_test)
#     cross_val_score(DT_classifier, train_X_prepared, y_train)
    
    
    
    RF_classifier = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1, random_state=0)
    RF_classifier.fit(train_X_prepared, y_train)
    train_score_RF = RF_classifier.score(train_X_prepared, y_train)
    train_score_RF = RF_classifier.score(test_X_prepared, y_test)
    
    return DT_classifier, RF_classifier, train_score_DT, test_score_DT, train_score_RF, train_score_RF
answer_three_a()

ModuleNotFoundError: No module named 'category_encoders'

(b) (10 points) For the Decision Tree Classifier, compute **feature importance** that comes with decision tree classifier in scikit-learn to get the top three most important features. Also do it for the random forest. Are they the same sets of features? Or do you have any interesting findings and comments?




In [167]:
DT_classifier, RF_classifier, _,_,_,_ = answer_three_a()
len(DT_classifier.feature_importances_),len(train_X_prepared), train_X_prepared.shape, X_train.shape
# len(RF_classifier.feature_importances_),len(X_train.columns)

/var/folders/cc/m0qhb3297hl_fmwwqxrf14c00000gn/T/ipykernel_735/1253243345.py:46: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  RF_classifier.fit(train_X_prepared, y_train)


(61, 749, (749, 61), (749, 20))

In [143]:
def answer_three_b():
    DT_classifier, RF_classifier, _,_,_,_ = answer_three_a()
    
    # your code here
    # there is a function which gives feature importance and another which visualises it
    DT_important_features = pd.Series(DT_classifier.feature_importances_,index=X_train.columns).sort_values(ascending=False).head(3)
#     DT = DT_classifier.feature_importances_[:8]
#     sorted_indices = np.argsort(DT_important_features)[::-1] 
#     labels = df.columns[1:]
#     print(DT_classifier.feature_names_in_)
    RF_important_features = RF_classifier.feature_importances_[:3]

    return DT_important_features, RF_important_features , sorted_indices, labels, DT
    # DT_important_features is np.array of 3 numbers indicates the indices of top 3 important features in the training features after onehot encoding.
    # DT_important_features is np.array of 3 numbers indicates the indices of top 3 important features in the training features after onehot encoding.
answer_three_b()

/var/folders/cc/m0qhb3297hl_fmwwqxrf14c00000gn/T/ipykernel_735/1453427024.py:46: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  RF_classifier.fit(train_X_prepared, y_train)


ValueError: Length of values (61) does not match length of index (20)

### Question 4 (20 points) Data Leakage
 
Suppose that you are helping the Ann Arbor Rock Music Club in organizing a concert in a week from now. They want you to predict the number of people who will attend this concert, so that they can arrange for drinks etc.
 
You have access to historical data about attendance in previous concerts, so you decide to design this as a regression problem with the following features:
 
1.   Day of the week
2.   Time of the day
3.   Number of tickets sold at the door
4.   Average quality of musicians performing
5.   Capacity of the venue
 
Your model performs well on training, validation and test data, so you are confident that it will successfully predict the attendance in the upcoming concert.
 
Is there a data leakage problem here? If so, what is the source?


In [9]:
# number of tickets sold at the door